# without CPU speeding up

In [3]:
import logging
from SALib.sample import saltelli
from SALib.analyze import sobol
import pandas as pd
import numpy as np
from tqdm import tqdm
from src.SugarScape import SugarScape
from src.Agents.Cell import Cell
from src.Agents.Trader import Trader

def setup_logger():
    logger = logging.getLogger("simulation")
    handler = logging.FileHandler("simulation.log")
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    return logger

def run_model(variable_parameters, map_scheme):
    logger = setup_logger()
    model = SugarScape(metabolism_mean=variable_parameters['metabolism_mean'], 
                       vision_mean=variable_parameters['vision'],
                       map_scheme=map_scheme)
    for step in range(max_steps):
        model.step()
        if step % 10 == 0:  # Output every 10 steps
            logger.info(f"Model step {step}: metabolism_mean={variable_parameters['metabolism_mean']}, vision={variable_parameters['vision']}, map_scheme={map_scheme}")
    total_population = model.schedule.get_agent_count()
    gini_coefficient = model.datacollector.get_model_vars_dataframe()['Gini'].values[-1]
    logger.info(f"Finished: metabolism_mean={variable_parameters['metabolism_mean']}, vision={variable_parameters['vision']}, total_population={total_population}, gini_coefficient={gini_coefficient}")
    return variable_parameters['metabolism_mean'], variable_parameters['vision'], total_population, gini_coefficient

# Define variables and their ranges
problem = {
    'num_vars': 2,
    'names': ['metabolism_mean', 'vision'],
    'bounds': [[2, 8], [2, 13]]
}

# Set experiment parameters
replicates = 10  # Simplify debugging
max_steps = 500  # Increase steps
distinct_samples = 8  # Set as 2^n

# Generate parameter samples
param_values = saltelli.sample(problem, distinct_samples)
total_runs = replicates * len(param_values)

# Create parameter list
param_list = []
for i in range(replicates):
    for vals in param_values:
        # Convert parameter sample values to integers
        variable_parameters = {name: int(val) for name, val in zip(problem['names'], vals)}
        param_list.append(variable_parameters)

# Initialize data storage
data_uniform = pd.DataFrame(index=range(total_runs), columns=['metabolism_mean', 'vision', 'Total Population', 'Gini Coefficient'])
data_random = pd.DataFrame(index=range(total_runs), columns=['metabolism_mean', 'vision', 'Total Population', 'Gini Coefficient'])
data_clustered = pd.DataFrame(index=range(total_runs), columns=['metabolism_mean', 'vision', 'Total Population', 'Gini Coefficient'])

# Run model in single process
map_schemes = ['uniform', 'top_heavy', 'split']
for map_scheme in map_schemes:
    results = []
    for params in tqdm(param_list, desc=f"Running for map_scheme={map_scheme}"):
        results.append(run_model(params, map_scheme))

    # Save results
    data = pd.DataFrame(results, columns=['metabolism_mean', 'vision', 'Total Population', 'Gini Coefficient'])
    data.to_csv(f'simulation_results_{map_scheme}.csv', index=False)

print("All experiments done!")



<ipython-input-3-11b053ba79dc>:47: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, distinct_samples)
Running for map_scheme=uniform:   0%|          | 0/480 [00:00<?, ?it/s]

Running for map_scheme=uniform:   0%|          | 0/480 [00:30<?, ?it/s]


KeyboardInterrupt: 

# with CPU speeding up
but not functioning well i think cuz the printing is stuck
i dont know why its stuck

In [ ]:
import logging
from SALib.sample import saltelli
from SALib.analyze import sobol
import pandas as pd
import numpy as np
from tqdm import tqdm
from src.SugarScape import SugarScape
from src.Agents.Cell import Cell
from src.Agents.Trader import Trader
import os

def setup_logger():
    """Set up a logger for each worker process."""
    logger = logging.getLogger("simulation")
    handler = logging.FileHandler("simulation.log")
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    return logger

def run_model(variable_parameters, map_scheme):
    """Run the SugarScape model with given parameters and log the process."""
    logger = setup_logger()
    model = SugarScape(metabolism_mean=variable_parameters['metabolism_mean'], 
                       vision_mean=variable_parameters['vision'],
                       map_scheme=map_scheme)
    for step in range(max_steps):
        model.step()
        if step % 10 == 0:  # Log every 10 steps
            logger.info(f"Model step {step}: metabolism_mean={variable_parameters['metabolism_mean']}, vision={variable_parameters['vision']}, map_scheme={map_scheme}")
    total_population = model.schedule.get_agent_count()
    gini_coefficient = model.datacollector.get_model_vars_dataframe()['Gini'].values[-1]
    logger.info(f"Finished: metabolism_mean={variable_parameters['metabolism_mean']}, vision={variable_parameters['vision']}, total_population={total_population}, gini_coefficient={gini_coefficient}")
    return variable_parameters['metabolism_mean'], variable_parameters['vision'], total_population, gini_coefficient

# Define the problem and its variable ranges
problem = {
    'num_vars': 2,
    'names': ['metabolism_mean', 'vision'],
    'bounds': [[1, 10], [2, 12]]
}

# Set experiment parameters
replicates = 10  # Number of replicates for each parameter set
max_steps = 20  # Number of steps in the model
distinct_samples = 16  # Number of distinct samples, set as a power of 2

# Generate parameter samples
param_values = saltelli.sample(problem, distinct_samples)
total_runs = replicates * len(param_values)
print('total_runs:', total_runs)

# Create a list of parameter sets
param_list = []
for i in range(replicates):
    for vals in param_values:
        # Convert parameter sample values to integers
        variable_parameters = {name: int(val) for name, val in zip(problem['names'], vals)}
        param_list.append(variable_parameters)
print('param_list:', param_list)

# Define map schemes
map_schemes = ['uniform', 'random', 'clustered']

# Run the model for each map scheme
for map_scheme in map_schemes:
    results = []
    for params in tqdm(param_list, desc=f"Running for map_scheme={map_scheme}"):
        results.append(run_model(params, map_scheme))

    # Store the results
    data = pd.DataFrame(results, columns=['metabolism_mean', 'vision', 'Total Population', 'Gini Coefficient'])
    data.to_csv(f'simulation_results_{map_scheme}.csv', index=False)

print("All experiments done!")



# Plotting

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from SALib.analyze import sobol
from itertools import combinations
import numpy as np

# Read experiment results
data = pd.read_csv('simulation_results.csv')

# Define variables and their ranges
problem = {
    'num_vars': 2,
    'names': ['metabolism_mean', 'vision'],
    'bounds': [[1, 10], [2, 12]]
}

# Perform Sobol sensitivity analysis
Si_population = sobol.analyze(problem, data['Total Population'].values, print_to_console=True)
Si_gini = sobol.analyze(problem, data['Gini Coefficient'].values, print_to_console=True)

# Visualization function for sensitivity indices
def plot_index(s, params, i, title=''):
    if i == '2':
        p = len(params)
        params = list(combinations(params, 2))
        indices = s['S' + i].reshape((p ** 2))
        # Filter indices to exclude NaN values
        indices = indices[~np.isnan(indices)]
        errors = s['S' + i + '_conf'].reshape((p ** 2))
        # Filter errors to exclude NaN values
        errors = errors[~np.isnan(errors)]
    else:
        indices = s['S' + i]
        errors = s['S' + i + '_conf']
        plt.figure()

    l = len(indices)
    plt.title(title)
    plt.ylim([-0.2, len(indices) - 1 + 0.2])
    plt.yticks(range(l), params)
    plt.errorbar(indices, range(l), xerr=errors, linestyle='None', marker='o')
    plt.axvline(0, c='k')

# Plot sensitivity indices
for Si, output_name in [(Si_population, "Total Population"), (Si_gini, "Gini Coefficient")]:
    for i in ['1', '2', 'T']:
        plot_index(Si, problem['names'], i, f'{output_name} - {i}-order sensitivity')
        plt.show()

